In [1]:
import xarray as xr

# Download

## Download SPEI from COPERNICUS project

Check the this [dataset](https://cds.climate.copernicus.eu/datasets/multi-origin-c3s-atlas?tab=overview) for mor infos, look for SPEI

In [ ]:
import cdsapi

dataset = "multi-origin-c3s-atlas"
request = {
    "origin": "cmip6",
    "experiment": "ssp5_8_5",
    "domain": "global",
    "period": "2015-2100",
    "variable": "monthly_standardised_precipitation_evapotranspiration_index_for_6_months_cumulation_period"
}

client = cdsapi.Client()
client.retrieve(dataset, request).download()


## Download raw variables

### Download pr

In [ ]:
%%bash 
cd ./data/pr
bash wget-20241222212617.sh -s
bash wget-20241222220936.sh -s
bash wget-20241222230917.sh -s

### Download tas

In [ ]:
%%bash 
cd ./data/tas
bash wget-20241222212252.sh -s
bash wget-20241222221111.sh -s
bash wget-20241222231101.sh -s

# Organizing files

In [12]:
import os

In [ ]:
EXPERIMENT ='ssp585'
variables = ['tas', 'pr']

for variable in variables:
    file_names = [file_name for file_name in os.listdir(f'./data/{variable}/') if file_name.endswith('.nc')]
    models = set([file_name.split('_')[2] for file_name in file_names])
    for model in models:
        _file_names = [file_name for file_name in file_names if model in file_name]
        os.makedirs(f'./data/{EXPERIMENT}/{variable}/{model}', exist_ok=True)
        for file_name in _file_names:
            os.system(f"mv ./data/{variable}/{file_name} ./data/{EXPERIMENT}/{variable}/{model}/{file_name}")


# Checking File Integrity and Dimensions

In [ ]:
# TODO: create scritp to check file ingetity

In [22]:
# checking datasets dimensions
variable = 'pr'
models = os.listdir(f'./data/{EXPERIMENT}/{variable}/')
models.sort()

for model in models:
    ds = xr.open_mfdataset(f'./data/{EXPERIMENT}/{variable}/{model}/*.nc')
    print(ds.sizes)
        

Frozen({'time': 1032, 'bnds': 2, 'lat': 324, 'lon': 432})
Frozen({'time': 1032, 'bnds': 2, 'lat': 192, 'lon': 384})
Frozen({'time': 3408, 'bnds': 2, 'lat': 192, 'lon': 288})


In [ ]:
# checking datasets dimensions
variable = 'tas'
models = os.listdir(f'./data/{EXPERIMENT}/{variable}/')
models.sort()

for model in models:
    ds = xr.open_mfdataset(f'./data/{EXPERIMENT}/{variable}/{model}/*.nc')
    print(ds.sizes)

Frozen({'time': 1032, 'bnds': 2, 'lat': 324, 'lon': 432})
Frozen({'time': 1032, 'bnds': 2, 'lat': 192, 'lon': 384})
Frozen({'time': 3408, 'bnds': 2, 'lat': 192, 'lon': 288})
